In [2]:
# Dependencies
import json

In [1]:
# Mike's code

In [ ]:
# Riley's code

In [15]:
category_json_path = "Resources/US_category_id.json"
category_ids = []
categories = []
indexer = 0

with open(category_json_path) as json_data:
  data = json.load(json_data)

items = data['items']

for item in items:
    print(data['items'][indexer]['id'])
    
    indexer = indexer + 1
        
category_ids

1
2
10
15
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44


[]

In [ ]:
# Jackie's code

In [ ]:
# Brian's code